In [3]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)


In [4]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data
import pandas as pd
import numpy as np


In [5]:
connection = crear_conexion()
reservas = consult_data(connection,'select * from plan_reservas')

Conexion Exitosa


In [6]:
connection = crear_conexion()
ordenes = consult_data(connection,'select orden,claot,equipo,material,felib,orden_texto,orden_estsis,reservasolped,orden_prio,orden_prio_texto,orden_revision,estatususuarioorden from plan_ordenes')

Conexion Exitosa


In [7]:
connection = crear_conexion()
aviso_orden = consult_data(connection,'select * from plan_avisorden')

Conexion Exitosa


In [8]:
connection = crear_conexion()
material = consult_data(connection,'select material,Cod_mate_antiguo,Grp_mat,Tp_mat,Jerarquia_produc from tbl_materialesIH09')

Conexion Exitosa


In [9]:
tabla_reservas = pd.DataFrame(reservas)
tabla_ordenes = pd.DataFrame(ordenes)
tabla_aviso = pd.DataFrame(aviso_orden)
tabla_material = pd.DataFrame(material)

In [10]:
tabla_aviso.drop_duplicates(['orden'],inplace=True)
tabla_ordenes.drop_duplicates(['orden'],inplace=True)

In [11]:
tabla_ordenes.rename(columns={"material": "modelo", "orden_prio": "prio_orden","orden_prio_texto" :"prio_orden_texto"},inplace=True)
tabla_aviso.drop(columns=['id','fechacarga'],inplace=True)
tabla_reservas.drop(columns=['id','fechacarga'],inplace=True)

In [12]:
#Se eliminar las reservas que se encuentran borradas

filtrar_tbl_reservas = tabla_reservas.loc[(tabla_reservas['salida_fin']!='X') & (tabla_reservas['reserva_pos_borrado']!='X') & (tabla_reservas['relevplnec']!='X') & (tabla_reservas['orden']!='')].reset_index(drop=True)

In [13]:
unir_reservas_ordenes = filtrar_tbl_reservas.merge(tabla_ordenes,how='left',on='orden')


In [14]:
unir_reservaxorden_aviso = unir_reservas_ordenes.merge(tabla_aviso,how='left',on='orden')
unir_reservaxorden_aviso.where(pd.notna(unir_reservaxorden_aviso),None,inplace=True)

In [15]:
unir_reservaxorden_aviso['felib'] = np.where( unir_reservaxorden_aviso['felib_x'].isnull(), unir_reservaxorden_aviso['felib_y']  , unir_reservaxorden_aviso['felib_x'])
unir_reservaxorden_aviso['equipo'] = np.where( unir_reservaxorden_aviso['equipo_x'].isnull(), unir_reservaxorden_aviso['equipo_y']  , unir_reservaxorden_aviso['equipo_x'])
unir_reservaxorden_aviso['orden_texto'] = np.where( unir_reservaxorden_aviso['orden_texto_x'].isnull(), unir_reservaxorden_aviso['orden_texto_y']  , unir_reservaxorden_aviso['orden_texto_x'])
unir_reservaxorden_aviso['orden_revision'] = np.where( unir_reservaxorden_aviso['orden_revision_x'].isnull(), unir_reservaxorden_aviso['orden_revision_y']  , unir_reservaxorden_aviso['orden_revision_x'])
unir_reservaxorden_aviso['estatususuarioorden'] = unir_reservaxorden_aviso['estatususuarioorden_x']
unir_reservaxorden_aviso['detalle01'] = unir_reservaxorden_aviso['estatususuarioorden_y']
unir_reservaxorden_aviso['detalle02'] = unir_reservaxorden_aviso['claot']

unir_reservaxorden_aviso.drop(columns='felib_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='felib_y', inplace=True)
unir_reservaxorden_aviso.drop(columns='equipo_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='equipo_y', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_texto_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_texto_y', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_revision_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_revision_y', inplace=True)

In [16]:
unir_reservaxorden_aviso['ate_accion'] = ""
unir_reservaxorden_aviso['ate_orden'] = 999
unir_reservaxorden_aviso['valor_flota'] = 0.00
unir_reservaxorden_aviso['stock_asignado'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c154'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c040'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c080'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c200'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c152'] = 0.00
unir_reservaxorden_aviso['trasl_fecha'] = None
unir_reservaxorden_aviso['Ped_nro'] = ''
unir_reservaxorden_aviso['Ped_Fecha'] = None
unir_reservaxorden_aviso['Ped_ctd'] = 0.00
unir_reservaxorden_aviso['Ped_proveedor'] = ''
unir_reservaxorden_aviso['estadoreserva'] = ''

In [17]:
unir_reservaxorden_aviso[['ctd_dif','eqvaloradq_usd']] = unir_reservaxorden_aviso[['ctd_dif','eqvaloradq_usd']].astype(float)


In [18]:
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['eqvaloradq_usd'].isna(),'eqvaloradq_usd'] = 1

In [19]:
unir_reservaxorden_aviso.loc[:,'orden_texto'] = unir_reservaxorden_aviso.loc[:,'orden_texto'].str.lstrip()

In [20]:
unir_reservaxorden_aviso['estadoreserva'] = np.where( (unir_reservaxorden_aviso['orden'] != "") & (unir_reservaxorden_aviso['ot_liberado'] == 'X') & (unir_reservaxorden_aviso['reserva_pos_borrado'] != 'X') ,"Lib Mov.Permitdo", unir_reservaxorden_aviso['estadoreserva'])
unir_reservaxorden_aviso['estadoreserva'] = np.where( (unir_reservaxorden_aviso['orden'] != "") & (unir_reservaxorden_aviso['ot_liberado'] == '') & (unir_reservaxorden_aviso['reservasolped'] == '3') & (unir_reservaxorden_aviso['reserva_pos_borrado'] != 'X') ,"Aprob Mov.Permitdo", unir_reservaxorden_aviso['estadoreserva'])
unir_reservaxorden_aviso['estadoreserva'] = np.where( (unir_reservaxorden_aviso['orden'] == "") & (unir_reservaxorden_aviso['reserva_pos_borrado'] != 'X') ,"Lib Mov.Permitdo", unir_reservaxorden_aviso['estadoreserva'])

In [21]:
# DETERMINAR ACCION de Abastecimiento
# 1. Revisamos los Servicios
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['um_base'] == "ZZ", 'ate_accion'] = "SERVICIO"
# 2. Revisamos las Reparaciones
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['lote'] == "REPARADO", 'ate_accion'] = "REPARADO"
# 3. Revisamos la Mercaderia
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['ate_accion'] == "", 'ate_accion'] = "MERCADERIA"
# 4. Anulamos los que no se deben atender: GARANTIA ubicado en el campo LOTE/Reservas Borradas/Reservas Atendidas o con Orden Cerrada campo salida_fin
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['salida_fin'] == "X" , 'ate_accion'] = "FIN"
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['reserva_pos_borrado'] == "X", 'ate_accion'] = "FIN"
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['lote'] == "GARANTIA" , 'ate_accion'] = "FIN"
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['reservasolped'] == "2" , 'ate_accion'] = "FIN"

In [22]:
# Personas -> EPPs y Uniformes
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] =='P','ate_orden'] = 100

In [23]:
# STC

# Equipos -> Ordenes de trabajo Libres o Aprovisionadas con Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 250, unir_reservaxorden_aviso['ate_orden'])

# OTROS -> 5er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo -- JOSE
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') , 310, unir_reservaxorden_aviso['ate_orden'])

# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 210, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 220, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0] == 'M'), 220, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 220, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 240, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo Libres o Aprovisionadas sin Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 300, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 260, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 270, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 270, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 270, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 290, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo Libres o Aprovisionadas sin Programa Semanal
# OTROS -> 5er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) & ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') , 930, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull() ) & ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['aviso'].isnull()) , 935, unir_reservaxorden_aviso['ate_orden'])


# Equipos -> Ordenes de trabajo NO Libres o NO Aprovisionadas con Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 950, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 910, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 920, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 920, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 920, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 940, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo NO Libres o NO Aprovisionadas sin Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 990, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 960, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 970, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 970, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 970, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'].isnull()) & ((unir_reservaxorden_aviso['reservasolped'].isnull()))) & (unir_reservaxorden_aviso['orden_revision'].isnull()) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 980, unir_reservaxorden_aviso['ate_orden'])


In [24]:
# Agrupar por Valor de Flota
lista_criterio1 = list(unir_reservaxorden_aviso["proyecto"])
lista_criterio2 = list(unir_reservaxorden_aviso["ate_orden"])

unir_reservaxorden_aviso['eqvaloradq_usd'] = unir_reservaxorden_aviso['eqvaloradq_usd'].replace('' , 0)

valor_flota = [unir_reservaxorden_aviso.loc[(unir_reservaxorden_aviso['proyecto'] == lista_criterio1[i]) & (unir_reservaxorden_aviso['ate_orden'] ==lista_criterio2[i]),"eqvaloradq_usd"].sum() for i in range(unir_reservaxorden_aviso.shape[0])]
# valor_flota.format("{:.2f}")
valor_flota = pd.DataFrame(valor_flota)
unir_reservaxorden_aviso["valor_flota"] = valor_flota

In [25]:
# STT
# ---

unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'].isnull())  & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller'), 999, unir_reservaxorden_aviso['ate_orden'])
# Equipos -> Ordenes de trabajo Libres con Programa Semanal
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación"), 230, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Preparación"), 230, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación venta"), 230, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Alistamiento"), 240, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo Libres sin Programa Semanal
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación"), 280, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Preparación"), 280, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación venta"), 280, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Alistamiento"), 290, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo NO Libres
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'].isnull())  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación"), 990, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'].isnull())  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Preparación"), 990, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'].isnull())  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación venta"), 990, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'].isnull())  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Alistamiento"), 990, unir_reservaxorden_aviso['ate_orden'])

# 2.3 Equipamiento y Otros

# Reservas para Centro de Costos
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['orden'].isnull()), 500, unir_reservaxorden_aviso['ate_orden'])
# Ordenes de Proyecto: 1 (113xxxx -> Proyectos de Mejora / 114xxxxx -> Compra de Activos / 18xxx -> Proyectos Campo) 
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] == "1" , 'ate_orden'] = 600
# Ordenes de Proyecto: 2
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] == "2" , 'ate_orden'] = 620
# Ordenes de Capacitacion: 7
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] == "7" , 'ate_orden'] = 640

# Las reservas que no se vana utilizar (ate_accion = "FIN"), colocar 999
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['ate_accion'] == "FIN" , 'ate_orden'] = 999
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['ate_orden'] == 999 , 'ate_accion'] = "FIN"

In [26]:
unir_reservaxorden_aviso['material'] = unir_reservaxorden_aviso['material'].str.replace(' ','')
tabla_material['material'] = tabla_material['material'].str.replace(' ','')

In [27]:
reservaxorden_material = unir_reservaxorden_aviso.merge(tabla_material,how='left',on='material')

In [40]:
reservaxorden_material['identificador']=(reservaxorden_material['orden'].fillna('')+reservaxorden_material['reserva']+reservaxorden_material['reservapos'])

In [41]:
reservaxorden_material = reservaxorden_material.sort_values(by = ['ate_orden', 'valor_flota','fe_nece', 'reserva', 'reservapos'], ascending = [True, False, True, True, True], na_position = 'last').reset_index(drop=True)
reservaxorden_material['index'] = reservaxorden_material.index

In [42]:
prueba = reservaxorden_material[['identificador','orden','aviso','ot_liberado','reservasolped','lote','reserva','reservapos','grupo_planif','ot_liberado','ate_orden','reservasolped','orden_revision','oden_clase_deno','orden_texto','ate_accion','ctd_dif']]

In [32]:
p = prueba.loc[prueba['ate_accion'] =='FIN']

In [33]:
p.to_excel('version02.xlsx')

In [30]:
p = prueba.loc[~(prueba['oden_clase_deno'] == 'Garantía')]

In [31]:
display(p.loc[(p['ate_accion']=='FIN') & (p['ctd_dif'] > 0) & (p['ot_liberado']=='X')])

,identificador,orden,reserva,reservapos,grupo_planif,ot_liberado,ate_orden,reservasolped,orden_revision,oden_clase_deno,orden_texto,ate_accion,ctd_dif
8824,4000011561252892,4000011561,25289,2,STC,X,999,3,None,Reparación Campo,OCO-RESUMEN DE ACEITE POR LA PTO,FIN,1.0
8867,4000005174103191,4000005174,10319,1,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-AVERIA DE MANIJA PUERTA TOLVA BKO827,FIN,1.0
8868,40000051961039811,4000005196,10398,11,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,1.0
8869,40000051961039812,4000005196,10398,12,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,10.0
8870,40000051961039813,4000005196,10398,13,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,10.0
8871,40000051961039814,4000005196,10398,14,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,5.0
8872,40000051961039815,4000005196,10398,15,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,10.0
8873,40000051961039817,4000005196,10398,17,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,2.0
8874,4000005196103982,4000005196,10398,2,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,1.0
8875,4000005196103985,4000005196,10398,5,STC,X,999,3,C7M27W24,Mantenimiento SSGG,OCO-ALISTAMIENTO ESTANDAR UNIDAD ANL932,FIN,1.0


In [28]:
selec_reservaxmaterial = reservaxorden_material[['index','identificador','material','ate_accion','ctd_dif','stock_asignado','trasl_ctd_c154','trasl_ctd_c040',
                                                   'trasl_ctd_c080','trasl_ctd_c200','trasl_ctd_c152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd',
                                                   'Ped_reserva','Ped_nro_necesidad','Ped_proveedor']]

In [29]:
filtrar_reservas = selec_reservaxmaterial.loc[(selec_reservaxmaterial['ate_accion']=='MERCADERIA') & (selec_reservaxmaterial['ctd_dif']>0)]